In [1]:
import dataset as ds
import utils
from torch.utils.data import DataLoader
import torch
import datetime
import transformer_timeseries as tst
import numpy as np

In [2]:
# Hyperparams
test_size = 0.1
batch_size = 128
target_col_name = "FCR_N_PriceEUR"
timestamp_col = "timestamp"
# Only use data from this date and onwards
cutoff_date = datetime.datetime(2017, 1, 1)

In [16]:
## Params
dim_val = 2048
n_heads = 8
n_decoder_layers = 4
n_encoder_layers = 4
dec_seq_len = 92 # length of input given to decoder
enc_seq_len = 153 # length of input given to encoder
output_sequence_length = 48 # target sequence length. If hourly data and length = 48, you predict 2 days ahead
window_size = enc_seq_len + output_sequence_length # used to slice data into sub-sequences
step_size = 1 # Step size, i.e. how many time steps does the moving window move at each step
in_features_encoder_linear_layer = 2048
in_features_decoder_linear_layer = 2048
max_seq_len = enc_seq_len
batch_first = False

In [17]:
# Define input variables
exogenous_vars = [] # should contain strings. Each string must correspond to a column name
input_variables = [target_col_name] + exogenous_vars
target_idx = 0 # index position of target in batched trg_y

input_size = len(input_variables)

In [18]:
# Read data
data = utils.read_data('dfs_merged_upload')

Reading file in data/dfs_merged_upload.csv


In [19]:
# Remove test data from dataset
training_data = data[:-(round(len(data)*test_size))]

In [20]:
# Make list of (start_idx, end_idx) pairs that are used to slice the time series sequence into chunkc.
# Should be training data indices only
training_indices = utils.get_indices_entire_sequence(
    data=training_data,
    window_size=window_size,
    step_size=step_size)

In [21]:
# Making instance of custom dataset class
training_data = ds.TransformerDataset(
    data=torch.tensor(training_data[input_variables].values).float(),
    indices=training_indices,
    enc_seq_len=enc_seq_len,
    dec_seq_len=dec_seq_len,
    target_seq_len=output_sequence_length
    )

From get_src_trg: data size = torch.Size([234, 1])


In [22]:
# Making dataloader
training_data = DataLoader(training_data, batch_size)

i, batch = next(enumerate(training_data))

src, trg, trg_y = batch

In [23]:
# Permute from shape [batch size, seq len, num features] to [seq len, batch size, num features]
if batch_first == False:

    shape_before = src.shape
    src = src.permute(1, 0, 2)
    print("src shape changed from {} to {}".format(shape_before, src.shape))

    shape_before = trg.shape
    trg = trg.permute(1, 0, 2)
    print("src shape changed from {} to {}".format(shape_before, src.shape))

src shape changed from torch.Size([33, 153, 1]) to torch.Size([153, 33, 1])
src shape changed from torch.Size([33, 48, 1]) to torch.Size([153, 33, 1])


In [24]:
model = tst.TimeSeriesTransformer(
    input_size=len(input_variables),
    dec_seq_len=enc_seq_len,
    batch_first=batch_first,
    dim_val=dim_val,
    num_predicted_features=1
    )

In [25]:
# Make src mask for decoder with size:
# [batch_size*n_heads, output_sequence_length, enc_seq_len]
src_mask = utils.generate_square_subsequent_mask(
    dim1=output_sequence_length,
    dim2=enc_seq_len
    )

In [26]:
# Make tgt mask for decoder with size:
# [batch_size*n_heads, output_sequence_length, output_sequence_length]
tgt_mask = utils.generate_square_subsequent_mask(
    dim1=output_sequence_length,
    dim2=output_sequence_length
    )

In [27]:
output = model(
    src=src,
    tgt=trg,
    src_mask=src_mask,
    tgt_mask=tgt_mask
    )

In [28]:
output

tensor([[[-0.0234],
         [ 0.1587],
         [-0.2655],
         ...,
         [-0.3121],
         [-0.5425],
         [ 0.2247]],

        [[-0.3787],
         [-0.4521],
         [-0.3991],
         ...,
         [-0.2042],
         [-0.1318],
         [-0.1057]],

        [[-0.2745],
         [-0.1126],
         [-0.4548],
         ...,
         [-0.4272],
         [ 0.1457],
         [ 0.0132]],

        ...,

        [[-0.0416],
         [-0.2546],
         [ 0.2775],
         ...,
         [-0.3728],
         [-0.0046],
         [ 0.0793]],

        [[-0.0755],
         [-0.2210],
         [-0.0999],
         ...,
         [-0.3648],
         [-0.0709],
         [-0.0254]],

        [[ 0.0583],
         [ 0.1233],
         [ 0.0472],
         ...,
         [-0.2169],
         [-0.3407],
         [-0.1757]]], grad_fn=<ViewBackward0>)